In [1]:
import torch
import random
from chess_env import ChessEnv

env = ChessEnv()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
for _ in range(1000):
    action = random.choice(env.action_space) # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)
    print(observation, reward, terminated, truncated, info)

    if terminated or truncated:
        observation, info = env.reset()

env.close()

1. f4 * 0 False False {'Move made': Move.from_uci('f2f4'), 'Reward': 0, 'Ended': False, 'Observed': '1. f4 *'}
1. f4 g5 * 0 False False {'Move made': Move.from_uci('g7g5'), 'Reward': 0, 'Ended': False, 'Observed': '1. f4 g5 *'}
1. f4 g5 2. h4 * 0 False False {'Move made': Move.from_uci('h2h4'), 'Reward': 0, 'Ended': False, 'Observed': '1. f4 g5 2. h4 *'}
1. f4 g5 2. h4 Na6 * 0 False False {'Move made': Move.from_uci('b8a6'), 'Reward': 0, 'Ended': False, 'Observed': '1. f4 g5 2. h4 Na6 *'}
1. f4 g5 2. h4 Na6 3. g4 * 0 False False {'Move made': Move.from_uci('g2g4'), 'Reward': 0, 'Ended': False, 'Observed': '1. f4 g5 2. h4 Na6 3. g4 *'}
1. f4 g5 2. h4 Na6 3. g4 e6 * 0 False False {'Move made': Move.from_uci('e7e6'), 'Reward': 0, 'Ended': False, 'Observed': '1. f4 g5 2. h4 Na6 3. g4 e6 *'}
1. f4 g5 2. h4 Na6 3. g4 e6 4. Nf3 * 0 False False {'Move made': Move.from_uci('g1f3'), 'Reward': 0, 'Ended': False, 'Observed': '1. f4 g5 2. h4 Na6 3. g4 e6 4. Nf3 *'}
1. f4 g5 2. h4 Na6 3. g4 e6 4. Nf